## Total Water Level
Notebook environment to migrate netcdf files to CF compliant zarr

In [49]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

### Set up modules and paths

In [50]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

<IPython.core.display.Javascript object>

In [51]:
# Project paths & files (manual input)
WP_dir = coclico_data_dir.joinpath("WP3")
data_dir = WP_dir.joinpath("data")
ds_dir = data_dir.joinpath("NetCDF")
ds_filename = ds_dir.joinpath("CTP_ReturnPeriods.nc")
ds_out_filename = ds_dir.joinpath("CTP_ReturnPeriods")

CF_dir = data_dir.joinpath(r"CF")

<IPython.core.display.Javascript object>

In [52]:
ds = xr.open_dataset(ds_filename)
# Add CRS
ds.attrs["crs"] = 3035
ds

<xarray.Dataset> Size: 2MB
Dimensions:  (locs: 51010)
Coordinates:
    lon      (locs) float64 408kB ...
    lat      (locs) float64 408kB ...
Dimensions without coordinates: locs
Data variables:
    RP1      (locs) float64 408kB ...
    RP100    (locs) float64 408kB ...
    RP1000   (locs) float64 408kB ...
Attributes:
    title:         EU Coastal Target Points - TWL Return Periods
    summary:       Location of Coastal Target Points and their respective TWL...
    institution:   Universidad de Cantabria, Instituto de Hidráulica Ambienta...
    keywords:      extreme event, hindcast, EU scale
    date_created:  04th of March 2024
    project:       CoCliCo - Coastal Climate Core Services
    Conventions:   CF-1.8
    crs:           3035

<IPython.core.display.Javascript object>

In [53]:
# Change locs to stations
ds = ds.rename_dims({'locs': 'stations'})

<IPython.core.display.Javascript object>

In [55]:
# Stack the data variables into a new dimension 'Return Periods'
rp_data = xr.concat([ds['RP1'], ds['RP100'], ds['RP1000']], dim='RP')

# Assign the new dimension labels
rp_data = rp_data.assign_coords({'RP': ['1', '100', '1000']})

# Now assign this data array back to the original dataset under a new variable name
ds['values'] = rp_data

# Optionally, you can remove the old data variables
ds = ds.drop_vars(['RP1', 'RP100', 'RP1000'])

ds

<xarray.Dataset> Size: 2MB
Dimensions:  (stations: 51010, RP: 3)
Coordinates:
    lon      (stations) float64 408kB ...
    lat      (stations) float64 408kB ...
  * RP       (RP) <U4 48B '1' '100' '1000'
Dimensions without coordinates: stations
Data variables:
    values   (RP, stations) float64 1MB 0.6883 0.6837 0.698 ... 0.7818 0.773
Attributes:
    title:         EU Coastal Target Points - TWL Return Periods
    summary:       Location of Coastal Target Points and their respective TWL...
    institution:   Universidad de Cantabria, Instituto de Hidráulica Ambienta...
    keywords:      extreme event, hindcast, EU scale
    date_created:  04th of March 2024
    project:       CoCliCo - Coastal Climate Core Services
    Conventions:   CF-1.8
    crs:           3035

<IPython.core.display.Javascript object>

In [56]:
rp_data

<xarray.DataArray 'RP1' (RP: 3, stations: 51010)> Size: 1MB
array([[0.68832815, 0.68372738, 0.6979847 , ..., 0.44570422, 0.45043558,
        0.44824767],
       [0.9168458 , 0.9123655 , 0.92665505, ..., 0.65814108, 0.6675154 ,
        0.66097295],
       [1.03718913, 1.0327723 , 1.04707873, ..., 0.77001584, 0.78183967,
        0.77300191]])
Coordinates:
    lon      (stations) float64 408kB -5.28 -5.269 -5.315 ... 14.57 14.57 14.58
    lat      (stations) float64 408kB 35.69 35.69 35.69 ... 35.87 35.85 35.86
  * RP       (RP) <U4 48B '1' '100' '1000'
Dimensions without coordinates: stations
Attributes:
    long_name:    1-yr return value of total water level, including tide, sur...
    short_name:   return_levels
    units:        m
    description:  From total water level hourly time series over the period 1...

<IPython.core.display.Javascript object>

In [57]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_filename, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [58]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_filename, working_dir=CF_dir)

# NOTE: File is already CF-compliant :)

{'P:\\11207608-coclico\\FULLTRACK_DATA\\WP3\\data\\NetCDF\\CTP_ReturnPeriods.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [59]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_filename), mode="w")

<IPython.core.display.Javascript object>

In [9]:
ds_out_filename

WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP3/data/NetCDF/CTP_ReturnPeriods')

<IPython.core.display.Javascript object>

In [62]:
ds_new = xr.open_zarr(ds_dir.joinpath("%s.zarr" % ds_out_filename))
ds_new

<xarray.Dataset> Size: 2MB
Dimensions:  (RP: 3, stations: 51010)
Coordinates:
  * RP       (RP) <U4 48B '1' '100' '1000'
    lat      (stations) float64 408kB dask.array<chunksize=(25505,), meta=np.ndarray>
    lon      (stations) float64 408kB dask.array<chunksize=(25505,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    values   (RP, stations) float64 1MB dask.array<chunksize=(2, 25505), meta=np.ndarray>
Attributes:
    Conventions:   CF-1.8
    crs:           3035
    date_created:  04th of March 2024
    institution:   Universidad de Cantabria, Instituto de Hidráulica Ambienta...
    keywords:      extreme event, hindcast, EU scale
    project:       CoCliCo - Coastal Climate Core Services
    summary:       Location of Coastal Target Points and their respective TWL...
    title:         EU Coastal Target Points - TWL Return Periods

<IPython.core.display.Javascript object>